In [4]:
import geojson
import pandas
import json
from shapely.geometry import shape, Point
from collections import OrderedDict
import csv
import numpy as np 

In [5]:
path = '../../TrentoTreeMap/data/geo_data_trees.geojson'

In [10]:
with open(path, 'r') as data:
    loaded_json = geojson.load(data)
    features = loaded_json['features']
    tree_names = set()
    for feature in features:
        properties = feature['properties']
        tree_names.add(properties['Name'])
    
    result = {}
    for tree in tree_names:
        result[tree] = {'count': 0, 'total_canopy_cover': 0}
        
    for feature in features: 
        properties = feature['properties']
        result[properties['Name']]['count'] += 1
        result[properties['Name']]['total_canopy_cover'] += float(properties['Canopy Cover (m2)'])
    
    sorted_dictionary = OrderedDict(sorted(result.items(), key=lambda item: item[1]['count'], reverse=True))
    
    result = []
    for key, value in sorted_dictionary.items():
        elem = sorted_dictionary[key]
        elem['mean_canopy_cover'] = elem['total_canopy_cover'] / elem['count']
        #elem.pop('total_canopy_cover')
        result += [[key, elem['count'], elem['mean_canopy_cover']]]
        
    with open('data1.csv', 'w+') as dump:
        writer = csv.writer(dump)
        writer.writerow(['name', 'count', 'mean_canopy_cover'])
        for line in result:
            writer.writerow(line)

In [11]:
circoscrizioni_path = '../../TrentoTreeMap/data/circoscrizioni.json'

In [22]:
df = pandas.read_csv('data1.csv')
top_n_tree_names = list(df['name'][:10])

with open(circoscrizioni_path, 'r') as c:
    with open(path, 'r') as data:
        loaded_json = geojson.load(data)
        circoscrizioni = geojson.load(c)
        result = {'SARDAGNA': {'Other': 1}, 'MEANO': {}, 'ARGENTARIO': {}, 'BONDONE': {}, 'MATTARELLO': {}, 'OLTREFERSINA': {}, 'POVO': {}, 
                  'RAVINA-ROMAGNANO': {}, 'S.GIUSEPPE-S.CHIARA': {}, 'VILLAZZANO': {}, 'GARDOLO': {}, 
                  'CENTRO STORICO PIEDICASTELLO': {}}
        
        for f in loaded_json['features']:
            try:
                point = Point(f['geometry']['coordinates'])        
                for feature in circoscrizioni['features']:
                    polygon = shape(feature['geometry'])
                    if polygon.contains(point):
                        tree_name = f['properties']['Name']
                        circoscrizione = feature['properties']['nome']
                        try:
                            if tree_name in top_n_tree_names:
                                curr_count = result[circoscrizione][tree_name]
                                result[circoscrizione][tree_name] += 1
                            else:
                                curr_count = result[circoscrizione]['Other']
                                result[circoscrizione]['Other'] += 1
                        except KeyError:
                            if tree_name in top_n_tree_names:
                                result[circoscrizione][tree_name] = 1
                            else:
                                result[circoscrizione]['Other'] = 1
            except TypeError:
                continue
        
        with open('data2.csv', 'w+') as dump:
            tree_names = set()
            for feature in loaded_json['features']:
                properties = feature['properties']
                tree_names.add(properties['Name'])
                
            writer = csv.writer(dump)
            headers = ['circoscrizioni'] + top_n_tree_names + ['Other']
            
            print(result)
            
            writer.writerow(headers)
        
            for k, v in result.items():
                to_write = [k]
                for nome_albero in headers[1:]:
                    try:
                        to_write += [result[k][nome_albero]]
                    except KeyError:
                        to_write += [0]
                
                writer.writerow(to_write)

{'SARDAGNA': {'Other': 33, 'Cupressus': 6}, 'MEANO': {'Other': 145, 'Celtis australis': 9, 'Cupressus': 20, 'Platanus x hispanica': 9, 'Sophora japonica': 1, 'Carpinus betulus': 2, 'Tilia cordata': 1, 'Tilia x europaea': 19}, 'ARGENTARIO': {'Other': 270, 'Acer campestre': 34, 'Platanus x hispanica': 12, 'Sophora japonica': 5, 'Aesculus hippocastanum': 9, 'Tilia x europaea': 10, 'Celtis australis': 61, 'Tilia cordata': 5, 'Cupressus': 25, 'Prunus cerasifera': 5, 'Carpinus betulus': 18}, 'BONDONE': {'Other': 187, 'Acer campestre': 9, 'Tilia cordata': 9, 'Cupressus': 23, 'Carpinus betulus': 3, 'Sophora japonica': 7, 'Tilia x europaea': 28, 'Platanus x hispanica': 2, 'Aesculus hippocastanum': 21, 'Celtis australis': 1, 'Prunus cerasifera': 2}, 'MATTARELLO': {'Other': 160, 'Carpinus betulus': 13, 'Cupressus': 25, 'Celtis australis': 25, 'Tilia cordata': 6, 'Sophora japonica': 13, 'Platanus x hispanica': 6, 'Tilia x europaea': 5, 'Aesculus hippocastanum': 1, 'Prunus cerasifera': 8, 'Acer cam

In [18]:
df = pandas.read_csv('data1.csv')
top_n_tree_names = list(df['name'][:10])

with open(circoscrizioni_path, 'r') as c:
    with open(path, 'r') as data:
        loaded_json = geojson.load(data)
        circoscrizioni = geojson.load(c)
        result = {'SARDAGNA': {'Other': 1}, 'MEANO': {}, 'ARGENTARIO': {}, 'BONDONE': {}, 'MATTARELLO': {}, 'OLTREFERSINA': {}, 'POVO': {}, 
                  'RAVINA-ROMAGNANO': {}, 'S.GIUSEPPE-S.CHIARA': {}, 'VILLAZZANO': {}, 'GARDOLO': {}, 
                  'CENTRO STORICO PIEDICASTELLO': {}}
        
        circoscrizioni_list = result.keys()
        
        result = {}
        for tree in top_n_tree_names: 
            result[tree] = {}
        
        result['Other'] = {}
        
        for f in loaded_json['features']:
            try:
                point = Point(f['geometry']['coordinates'])        
                for feature in circoscrizioni['features']:
                    polygon = shape(feature['geometry'])
                    if polygon.contains(point):
                        tree_name = f['properties']['Name']
                        circoscrizione = feature['properties']['nome']
                        try:
                            if tree_name in top_n_tree_names:
                                curr_count = result[tree_name][circoscrizione]
                                result[tree_name][circoscrizione] += 1
                            else:
                                curr_count = result['Other'][circoscrizione]
                                result['Other'][circoscrizione] += 1
                        except KeyError:
                            if tree_name in top_n_tree_names:
                                result[tree_name][circoscrizione] = 1
                            else:
                                result['Other'][circoscrizione] = 1
            except TypeError:
                continue
        
        with open('data3.csv', 'w+') as dump:
            tree_names = set()
            for feature in loaded_json['features']:
                properties = feature['properties']
                tree_names.add(properties['Name'])
                
            writer = csv.writer(dump)
            headers = ['tree_name'] 
            for i in circoscrizioni_list:
                headers.append(i)
                
            writer.writerow(headers)
            
            print(result)
            
            for k, v in result.items():
                to_write = [k]
                for c in circoscrizioni_list:
                    try:
                        to_write += [result[k][c]]
                    except KeyError:
                        to_write += [0]
                writer.writerow(to_write)

            '''
            for k, v in result.items():
                to_write = [k]
                for nome_albero in headers[1:]:
                    try:
                        to_write += [result[nome_albero][k]]
                    except KeyError:
                        to_write += [0]
                
                writer.writerow(to_write)
            '''

{'Celtis australis': {'VILLAZZANO': 19, 'POVO': 21, 'MATTARELLO': 25, 'RAVINA-ROMAGNANO': 15, 'GARDOLO': 211, 'MEANO': 9, 'ARGENTARIO': 61, 'CENTRO STORICO PIEDICASTELLO': 242, 'OLTREFERSINA': 341, 'S.GIUSEPPE-S.CHIARA': 83, 'BONDONE': 1}, 'Aesculus hippocastanum': {'ARGENTARIO': 9, 'S.GIUSEPPE-S.CHIARA': 405, 'CENTRO STORICO PIEDICASTELLO': 91, 'OLTREFERSINA': 34, 'GARDOLO': 4, 'RAVINA-ROMAGNANO': 3, 'BONDONE': 21, 'POVO': 13, 'MATTARELLO': 1}, 'Carpinus betulus': {'BONDONE': 3, 'RAVINA-ROMAGNANO': 5, 'MATTARELLO': 13, 'VILLAZZANO': 33, 'GARDOLO': 53, 'CENTRO STORICO PIEDICASTELLO': 159, 'OLTREFERSINA': 126, 'S.GIUSEPPE-S.CHIARA': 123, 'ARGENTARIO': 18, 'MEANO': 2, 'POVO': 1}, 'Tilia cordata': {'VILLAZZANO': 35, 'RAVINA-ROMAGNANO': 34, 'POVO': 1, 'BONDONE': 9, 'MATTARELLO': 6, 'GARDOLO': 216, 'ARGENTARIO': 5, 'S.GIUSEPPE-S.CHIARA': 31, 'OLTREFERSINA': 116, 'CENTRO STORICO PIEDICASTELLO': 55, 'MEANO': 1}, 'Platanus x hispanica': {'POVO': 2, 'GARDOLO': 169, 'ARGENTARIO': 12, 'S.GIUSEPPE

In [16]:
with open(path, 'r') as data:
    loaded_json = geojson.load(data)
    features = loaded_json['features']
    
    result = []
    
    for feature in features: 
        properties = feature['properties']
        result.append([properties['Height (m)']])
    
    with open('data4.csv', 'w+') as dump:
        writer = csv.writer(dump)
        writer.writerow(['height'])
        for line in result:
            writer.writerow(line)

In [23]:
with open(path, 'r') as data:
    df = pandas.read_csv('data1.csv')
    top_n_tree_names = list(df['name'][:10])

    loaded_json = geojson.load(data)
    features = loaded_json['features']
    
    result = []
    for feature in features:
        properties = feature['properties']
        if properties['Name'] in top_n_tree_names:
            result.append([properties['Name'], properties['Height (m)'], properties['Gross Carbon Sequestration (kg/yr)'], properties['Canopy Cover (m2)']])

    with open('data5.csv', 'w+') as dump:
        writer = csv.writer(dump)
        writer.writerow(['name', 'height', 'co2_absorption', 'canopy_cover'])
        for line in result:
            writer.writerow(line)

In [28]:
# data 7 creation
with open('../assets/data7.csv', 'w+') as dump:
    df = pandas.read_csv('data2.csv')
    circ_names = list(df.circoscrizione)
    n_trees = list(df.sum(axis = 1))
    #print(circ_names)
    #print(list(df.sum(axis = 1)))
    result = []
    for i, circ in enumerate(circ_names):
        result.append([circ, circ, n_trees[i], 0])
    #print(result)

    writer = csv.writer(dump)
    writer.writerow(['name', 'code', 'pop', 'area'])
    for line in result:
        writer.writerow(line)

C:\Users\ricca\AppData\Local\Temp\ipykernel_12828\461144886.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  n_trees = list(df.sum(axis = 1))
